## Expense Automation
This goes through previous expense reports and automatically fills in the new expense report form.

In [39]:
# importing libraries
import pandas as pd
import os
import openpyxl

Reading in the old files.

In [40]:
dfs = []
expense_pars = []
for file in os.listdir('previous_companies'):
    # if the file is a csv file
    if file.endswith('.csv'):
        # read the csv file
        df = pd.read_csv('previous_companies/' + file)
        # add the company name to the dataframe
        df['company'] = file.split('.')[0]
        # append the dataframe to the list of dataframes
        dfs.append(df)
    # read .xlsx files using the same method
    elif file.endswith('.xlsx'):
        df = pd.read_excel('previous_companies/' + file)
        df['company'] = file.split('.')[0]
        dfs.append(df)

Cleaning data.

In [41]:
# making dfs one dataframe
df = pd.concat(dfs)
df

,Line Item,Category,company
0,6005.1 • Administrator,Nursing Administration,company_a
1,6005.1 • Activities - Wages,Activities And Social Services,company_a
2,6005.2 • ALA,Nursing Care,company_a
3,6005.21 • Clerical,Nursing Administration,company_a
4,6005.3 • Cooks,Other Dietary,company_a
0,6005 • Salaries & Wages,G&A,company_b
1,6015.1 • Food Purchases,Raw Food,company_b
2,7180 • Consulting,Addback Expense,company_b
3,7181 • Contract Labor - Third Party,Agency,company_b


Taking substring of Line Items (i.e. removing #s). Only run this code chunk once or else you'll get errors.

In [42]:
# splitting line item by • and adding it to a new column
df['cleaned_line_item'] = df['Line Item'].str.split('•').str[-1]
# removing leading and trailing spaces
df['cleaned_line_item'] = df['cleaned_line_item'].str.strip()
# deleting the old line item column
del df['Line Item']

In [44]:
# making sure it worked
df

,Category,company,cleaned_line_item
0,Nursing Administration,company_a,Administrator
1,Activities And Social Services,company_a,Activities - Wages
2,Nursing Care,company_a,ALA
3,Nursing Administration,company_a,Clerical
4,Other Dietary,company_a,Cooks
0,G&A,company_b,Salaries & Wages
1,Raw Food,company_b,Food Purchases
2,Addback Expense,company_b,Consulting
3,Agency,company_b,Contract Labor - Third Party


Scraping old data for using Line Item / Category pairs.

In [50]:
# read through df and add unique pairs of category and line item to expense_pars
expense_pars = []
for index, row in df.iterrows():
    if (row['Category'], row['cleaned_line_item']) not in expense_pars:
        expense_pars.append((row['cleaned_line_item'], row['Category']))


In [51]:
expense_pars

[('Administrator', 'Nursing Administration'),
 ('Activities - Wages', 'Activities And Social Services'),
 ('ALA', 'Nursing Care'),
 ('Clerical', 'Nursing Administration'),
 ('Cooks', 'Other Dietary'),
 ('Salaries & Wages', 'G&A'),
 ('Food Purchases', 'Raw Food'),
 ('Consulting', 'Addback Expense'),
 ('Contract Labor - Third Party', 'Agency')]

Taking in a new company data.

In [56]:
new_co = pd.read_excel('new_company.xlsx')
new_co

,Line Item,Category
0,34 • Administrator,NaN
1,9.1.3213 • Activities - Wages,NaN
2,8905 • Salaries & Wages,NaN
3,60 • Food Purchases,NaN
4,60 • Beer Purchases,NaN
5,60 • Decorations,NaN


In [57]:
new_co

,Line Item,Category
0,34 • Administrator,NaN
1,9.1.3213 • Activities - Wages,NaN
2,8905 • Salaries & Wages,NaN
3,60 • Food Purchases,NaN
4,60 • Beer Purchases,NaN
5,60 • Decorations,NaN


In [60]:
# splitting line item by • and adding it to a new column
new_co['cleaned_line_item'] = new_co['Line Item'].str.split('•').str[-1]
# removing leading and trailing spaces
new_co['cleaned_line_item'] = new_co['cleaned_line_item'].str.strip()
# deleting the old line item column
del new_co['Line Item']

Transmitting learnings from old data to new_co.

In [61]:
# for loop that goes through each line item in new_co and check if the line item is in expense_pars 'cleaned_line iterm'. If it is, take the category from expense_pars and add it to the new_co dataframe
for index, row in new_co.iterrows():
    for expense_par in expense_pars:
        if row['cleaned_line_item'] == expense_par[0]:
            new_co.loc[index, 'Category'] = expense_par[1]
            

In [64]:
# switching the order of the columns in new_co
new_co = new_co[['cleaned_line_item', 'Category']]
new_co

,cleaned_line_item,Category
0,Administrator,Nursing Administration
1,Activities - Wages,Activities And Social Services
2,Salaries & Wages,G&A
3,Food Purchases,Raw Food
4,Beer Purchases,NaN
5,Decorations,NaN


Writing to a new csv.

In [65]:
# write the new_co dataframe to a new excel file
new_co.to_excel('new_company_cleaned.xlsx', index=False)